In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from PIL import Image
import os
import re
import warnings

warnings.filterwarnings('ignore')


In [6]:
df = pd.read_csv('all_cells_tpm.csv')
print(df.columns)

# Here are the things you change.
mode = 'name'	# can be Summary, name, symbol, GO (caps matter)
search_term = 'clock'	# not case sensititve
max_stitch = 10		# number of plots per row 
outputName = 'image1.jpg'

imageFolder = 'images/'
tempDir = 'temp/'
groups = ['slow', 'fast', '22c06_', 'PF', 'AC']


Index(['Unnamed: 0', 'ID', 'slow1', 'slow2', 'slow3', 'fast1', 'fast2',
       'fast3', '22c06_1', '22c06_2', '22c06_3', '22c06_4', 'PF1', 'PF2',
       'PF3', 'PF4', 'AC1', 'AC2', 'AC3', 'name', 'symbol', 'Summary', 'GO'],
      dtype='object')


In [7]:
extract_groups = [[t for t in df.columns if g in t] for g in groups]
fb_list = []
# Find search term in mode and add to list
for ind in df.index:
    try:
        if re.search(search_term, df[mode][ind], re.IGNORECASE):
            fb_list.append(df['ID'][ind])
    except:
        pass
    
print(f'There are {len(fb_list)} matching terms.')

# Plot each TPM for each gene
for k,fb in enumerate(fb_list):
    test = [np.array(df[df.ID == fb][g]).flatten() for g in extract_groups]
    g = sns.boxplot(data=test)
    g.set_xticklabels(groups)
    g.set_title(df[df.ID == fb].name.item())
    plt.savefig(tempDir+'Test'+str(k)+'.jpg')
    plt.close()

# stitch images together
image_list = [tempDir +'Test'+str(k)+'.jpg' for k in range(len(fb_list))]
tot_images = len(image_list)

images = [Image.open(x) for x in image_list]
width, height = images[0].size

total_width = max_stitch*width
max_height = (tot_images//max_stitch + 1*(tot_images%max_stitch>0))*height

new_im = Image.new('RGB', (total_width, max_height))

for k,im in enumerate(images):
    x,y = (k//max_stitch),(k%max_stitch)
    new_im.paste(im, (y*width,x*height))

new_im.save(imageFolder+outputName)


There are 3 matching terms.
